In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorica

In [2]:
import sys
sys.argv = [""]

In [29]:
from cart_pole.utils import initialize
from cart_pole.agent import Agent
from numpy.random import Generator, MT19937, SeedSequence
from torch.utils.tensorboard import SummaryWriter
from cart_pole.environment import make_sync_vector_env

In [30]:
seed_seq = SeedSequence(args.seed)
rng = Generator(MT19937(seed_seq))

In [31]:
args, writer = initialize(seconds_since_epoch=1764177877)
envs = make_sync_vector_env(args)

In [33]:
agent = Agent(envs).to(args.device)

optimizer = optim.Adam(agent.parameters(),
                       lr=args.learning_rate,
                       eps=1e-5)

In [90]:
class Storage(object):

    def __init__(self,
                 args):

        self.num_steps = args.num_steps
        self.device = args.device

        self.reset()

    def store(self,
              paramid,
              value):

        assert self.step < self.num_steps, "Storage is full"
        self.__dict__[paramid][self.step] = value

    def next_step(self):
        self.step += 1

    def reset(self):

        self.step = 0
        
        self.obs =\
            torch.zeros((args.num_steps, args.num_envs) +
                        envs.single_observation_space.shape).to(self.device)
    
        self.actions =\
            torch.zeros((args.num_steps, args.num_envs) +
                        envs.single_action_space.shape).to(self.device)

        self.logprobs =\
            torch.zeros((args.num_steps, args.num_envs)).to(self.device)
    
        self.rewards =\
            torch.zeros((args.num_steps, args.num_envs)).to(self.device)
    
        self.terminated =\
            torch.zeros((args.num_steps, args.num_envs)).to(self.device)

        self.truncated =\
            torch.zeros((args.num_steps, args.num_envs)).to(self.device)

        self.values =\
            torch.zeros((args.num_steps, args.num_envs)).to(self.device)

storage = Storage(args)

In [37]:
seeds = [int(rng.uniform(0, 1E9)) for _ in np.arange(args.num_envs)]
envs.reset(seed=seeds)

In [ ]:
global_step = 0
start_time = time.time()

In [89]:
next_obs = torch.Tensor(envs.reset(seed=seeds)[0]).to(args.device)
next_terminated = torch.zeros(args.num_envs).to(args.device)
next_truncated = torch.zeros(args.num_envs).to(args.device)
num_updates = args.total_timesteps // args.batch_size
storage.reset()

for _ in range(1):

    storage.store("obs", next_obs)
    storage.store("terminated", next_terminated)
    storage.store("truncated", next_truncated)

    with torch.no_grad():
        action, logprob, _, value = agent.get_action_and_value(next_obs)
        storage.store("values", value.flatten())
        storage.store("actions", action)
        storage.store("logprobs", logprob)

    next_obs, reward, terminated, truncated, info =\
        envs.step(action.cpu().numpy())

    reward = torch.tensor(reward).to(args.device).view(-1)
    next_obs = torch.Tensor(next_obs).to(args.device)
    next_terminated = torch.Tensor(terminated).to(args.device)
    next_truncated = torch.Tensor(truncated).to(args.device)

    for item in info:
        if "episode" in item.keys():
            print(f"global_step={global_step}," +\
                  "episodic_return={item['episode']['r']}")

In [82]:
reward

array([1., 1., 1., 1.])